https://apps.bea.gov/iTable/iTable.cfm?reqid=147&step=51&isuri=1&table_list=13

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrsrm = {'I': 1, 'II': 4, 'III': 7, 'IV': 10}

In [7]:
s = ['Quarter', 'Government', 'Utilities', 'Wholesale trade', 'Retail trade', 
     'Transportation and warehousing', 'Private goods-producing industries [1]',
     'Private services-producing industries [2]',
     'Finance, insurance, real estate, rental, and leasing',
     'Educational services, health care, and social assistance', 
     'Construction', 'Manufacturing']

In [9]:
d = {}
for dset in [1, 2, 3]:
    # Adjust top and bottom line 1, 2, 3 (downloaded data manually)
    df = pd.read_csv(f'raw/gdpbyindustry{dset}.csv', skiprows=4).T

    columns = df.iloc[1].str.strip()
    if dset == 3:
        columns[0] = 'Quarter'

    df.columns = columns

    df = df.iloc[2:].loc[:, s]

    if dset == 3:
        dates = list(zip([dt[:4] for dt in df.index], df.Quarter.map(qtrsrm).values))
        date_index = pd.to_datetime([f'{year}-{month}-01' for year, month in dates])

        df = (df.reset_index().drop(['index', 'Quarter'], axis=1)
                .set_index(date_index).astype(float))

    else:
        dates = pd.to_datetime([f'{year}-01-01' for year in df.index])
        df = (df.reset_index().drop(['index', 'Quarter'], axis=1)
                .set_index(dates).astype(float))    

    df.index.name = 'date'
    df['TTU'] = df[['Utilities', 'Wholesale trade', 'Retail trade', 
                    'Transportation and warehousing']].sum(axis=1)

    df['Oth_Serv'] = (df['Private services-producing industries [2]'] - df['TTU']
                      - df['Finance, insurance, real estate, rental, and leasing'])

    df['FIRE'] = df['Finance, insurance, real estate, rental, and leasing']

    df['GoodsTTU'] = df['Private goods-producing industries [1]'] + df['TTU']

    d[dset] = df

In [ ]:
(d[1].append(d[2]).loc['1989':'2005'].resample('QS').pad().iloc[:-1]
    .append((d[1].append(d[2]).loc['2005'] - (d[3].loc['2005'].sum() / 3)).append(d[3]))
    .loc[:,['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU']]).to_csv(data_dir / 'gdpva.csv', index_label='date')

In [18]:
r = (d[1].append(d[2]).loc['1989':'2005'].resample('QS').pad().iloc[:-1]
    .append((d[1].append(d[2]).loc['2005'] - (d[3].loc['2005'].sum() / 3)).append(d[3])))

(r[['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU']]
 .to_csv(data_dir / 'gdpva.csv', index_label='date'))

In [20]:
(r[['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU', 
    'Educational services, health care, and social assistance', 
    'Construction', 'Manufacturing', 'Retail trade']]
 .to_csv(data_dir / 'gdpvafull.csv', index_label='date'))

In [12]:
d[1].append(d[2])

Unnamed: 1,Government,Utilities,Wholesale trade,Retail trade,Transportation and warehousing,Private goods-producing industries [1],Private services-producing industries [2],"Finance, insurance, real estate, rental, and leasing","Educational services, health care, and social assistance",Construction,Manufacturing,TTU,Oth_Serv,FIRE,GoodsTTU
date,,,,,,,,,,,,,,,
1987-01-01,0.39,0.27,-0.20,0.02,0.14,1.33,1.44,0.30,0.35,0.13,1.10,2.300000e-01,0.91,0.30,1.56
1988-01-01,0.42,-0.10,0.32,0.63,0.10,1.29,2.53,0.51,0.08,0.16,1.13,9.500000e-01,1.07,0.51,2.24
1989-01-01,0.39,0.21,0.44,0.27,0.11,0.19,2.92,0.48,0.22,0.04,0.06,1.030000e+00,1.41,0.48,1.22
1990-01-01,0.36,0.12,-0.03,0.07,0.11,-0.09,1.53,0.33,0.20,-0.07,-0.10,2.700000e-01,0.93,0.33,0.18
1991-01-01,0.07,-0.08,0.31,-0.02,0.11,-0.46,0.33,0.30,0.15,-0.30,-0.21,3.200000e-01,-0.29,0.30,-0.14
1992-01-01,0.13,0.03,0.46,0.29,0.21,0.81,2.51,0.67,0.14,0.05,0.65,9.900000e-01,0.85,0.67,1.80
1993-01-01,0.03,-0.04,0.21,0.40,0.13,0.59,2.04,0.51,0.15,0.11,0.59,7.000000e-01,0.83,0.51,1.29
1994-01-01,0.08,0.04,0.33,0.48,0.25,1.50,2.37,0.45,0.10,0.20,0.96,1.100000e+00,0.82,0.45,2.60
1995-01-01,0.01,0.03,-0.01,0.32,0.09,0.48,1.75,0.53,0.06,0.04,0.63,4.300000e-01,0.79,0.53,0.91


In [6]:
pd.read_csv(f'raw/gdpbyindustry{dset}.csv', skiprows=4)

,Line,Unnamed: 1,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997
0,1,Percent change:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Gross domestic product,3.5,4.2,3.7,1.9,-0.1,3.6,2.7,4.0,2.7,3.8,4.5
2,3,Percentage points:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Private industries,2.77,3.82,3.10,1.44,-0.13,3.31,2.63,3.87,2.23,4.01,4.71
4,5,"Agriculture, forestry, fishing, and hunting",0.07,-0.18,0.16,0.09,0.00,0.16,-0.18,0.24,-0.23,0.14,0.12
5,6,Farms,0.01,-0.14,0.16,0.09,0.00,0.18,-0.14,0.22,-0.23,0.15,0.12
6,7,"Forestry, fishing, and related activities",0.05,-0.04,0.00,0.00,0.00,-0.01,-0.03,0.02,0.00,-0.01,-0.01
7,8,Mining,0.03,0.18,-0.07,-0.01,0.05,-0.05,0.07,0.11,0.03,-0.07,0.05
8,9,Oil and gas extraction,0.00,0.07,-0.06,-0.04,0.02,-0.05,0.01,0.07,0.06,-0.09,0.00
9,10,"Mining, except oil and gas",0.00,0.09,0.01,0.02,0.03,0.00,0.01,0.04,-0.01,0.03,0.02


In [131]:
(df1.append(df2).loc['1989':'2005'].resample('QS').pad().iloc[:-1]
    .append((df1.append(df2).loc['2005'] - (df3.loc['2005'].sum() / 3)).append(df3))
    .loc[:,['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU']]).to_csv(data_dir / 'gdpva.csv', index_label='date')